In [1]:
import heapq
from IPython.display import clear_output
import io
import math
TAMAÑO_BLOQUE= 48
BYTESVENTANA = 4
BLOCK_SIZE= 10

    Aplicamos la transformada de Burrows-Wheeler 
    

In [9]:


def insert_shift(string):
    string = bytearray(string)
    ultimo_byte = string.pop()
    string.insert(0, ultimo_byte)
    return string



def find_index(headquarter, string):
    return headquarter.index(string)

def obtain_bwt(headquarter):
    output =[]
    for i in headquarter:
        output.append(i[-1])
    print(bytes(output))
    return output

def sort_headquarters(headquarters):
  heapq.heapify(headquarters)
  return [heapq.heappop(headquarters) for _ in range(len(headquarters))]

def bwt(string):
    headquarter = list()
    for i in range(len(string)):
        string = insert_shift(string)
        headquarter.append(string)
        headquarter= sort_headquarters(headquarter)
    return (obtain_bwt(headquarter), find_index(headquarter, string))

def format_header_and_indexes(byte):
    if(len(byte) < BYTESVENTANA):
        byte_a_agregar=BYTESVENTANA-len(byte)
        #print("resultado: ",byte,"\nCantidad de bytes",len(byte))
        resultado = bytes([0] * byte_a_agregar)+byte
        #print("resultado: ",resultado,"\nCantidad de bytes",len(resultado),"\nEn numero: ",int.from_bytes(resultado, "big"))
        return resultado
    elif( len(byte) > BYTESVENTANA):
        print("tamaño de la ventana no admisible")

def save_header(cut_point, filename):
    with open(filename+".bwt","ab") as file:
        tamaniosegmento = cut_point
        tamaniosegmentobytes = tamaniosegmento.to_bytes((tamaniosegmento.bit_length() + 7) // 8, "big")
        file.write(format_header_and_indexes(tamaniosegmentobytes))
def save_data(bwt_data, filename):
    with open(filename+".bwt", "ab") as file:
        #print(bwt_data)
        indicebytes = int(bwt_data[1]).to_bytes((int(bwt_data[1]).bit_length() + 7) // 8, "big")
        file.write(format_header_and_indexes(indicebytes))
        file.write(bytes(bwt_data[0]))
        file.close()

def bwt_from_bytes(filename):
    with open(filename, "rb") as f:
        blocks = []
        data = f.read()
        print(int(len(data)//TAMAÑO_BLOQUE)+1)
        for i in range(int(len(data)//TAMAÑO_BLOQUE)+1):
            if i < int(len(data)//TAMAÑO_BLOQUE):    
                clear_output(wait=True)
                print(i*TAMAÑO_BLOQUE,":",(i+1)*TAMAÑO_BLOQUE,"     ",((i+1)*TAMAÑO_BLOQUE/len(data))*100,"%")
                clean_data = data[i*TAMAÑO_BLOQUE:(i+1)*TAMAÑO_BLOQUE]
                bwt_data = bwt(clean_data)
            else:
                print("insertando el ultimo bloque\ncantidad de bytes del bloque: ",len(data[i*TAMAÑO_BLOQUE:]),"\nbytes restantes para completar el bloque: ",TAMAÑO_BLOQUE-len(data[i*TAMAÑO_BLOQUE:]))
                bytes_a_agregar = TAMAÑO_BLOQUE-len(data[i*TAMAÑO_BLOQUE:])
                ultimo_bloque = data[i*TAMAÑO_BLOQUE:]
                bwt_data = bwt(ultimo_bloque)
                bwt_data = (bytes(bwt_data[0]) + bytes(bytes_a_agregar),bwt_data[1])
            blocks.append(bwt_data)
        save_header(TAMAÑO_BLOQUE-bytes_a_agregar, filename)
        for b in blocks:
            save_data(b,filename)
    print("Se guardo el archivo")
    return bwt_data
def bwt_show(filename):
    with open(filename,"rb") as f:
        data = f.read()
        print(data)
        punto_de_corte=int.from_bytes(data[:BYTESVENTANA], "big")
        indices=int.from_bytes(data[BYTESVENTANA:BYTESVENTANA+BYTESVENTANA], "big")
        bloques=((len(data)-BYTESVENTANA))/(BYTESVENTANA+TAMAÑO_BLOQUE)
        print((BYTESVENTANA+TAMAÑO_BLOQUE))
        print((len(data)-BYTESVENTANA))
        print("bytes validos del ultimo bloque: ",punto_de_corte,"\nindice 1: ",indices,"\nCantidad de bloques: ",bloques, "\nTamaño del archivo: ",len(data))
        

if __name__ == "__main__":
    filename = "../datos/texto1.txt"
    bwt_data = bwt_from_bytes(filename)
    bwt_show(filename+".bwt")


10176 : 10224       99.96089166992569 %
b'a.\xb3aeael onr  r  dc arlasmvauc  Eo edsjeee  fo\xc3c'
insertando el ultimo bloque
cantidad de bytes del bloque:  4 
bytes restantes para completar el bloque:  44
b'\xad.\xc3m'
Se guardo el archivo
b"\x00\x00\x00\x04\x00\x00\x00\t,s6reloo n lru e etrddinctecae  uaahpdmoulen mmD\x00\x00\x00\x14ssoon,e\xadhrt\xadrlb ianapr d  ll aaaedd bgtea an\xc3\xc3sb\x00\x00\x00\x12.noaaoao\xb3 nl\xadeo itu nmbruU idsrtr ee\xbae  p \xc3\xc3\xc3dnp\x00\x00\x00%ssl\xb3esarrpln osdnnarep ioatcrpdlo   ot peoo  e\xc3g\x00\x00\x00\x08lsaaoseecm  o  noud uipeealloedttrcdlm arooo  qq\x00\x00\x00\x0cd,eeaaa\x9c tdb\xe2rhameanatud p SlBoi nd beewssfo\xe2\x80s \x00\x00\x00.oynrorn\x9dro a tpdsr vv ueietdspls ioebe enn ie o\x80\x00\x00\x00&,,,s,sssssrmti   dtcl rnleaueoamifnrceyooioae h \x00\x00\x00\x03nssunsoaar lpjin   adrfsicdde ouedri aeaeuao   s\x00\x00\x00\x1fe,aaaaannzvltnnas uu u  ncoe\xb3u e sier esffn er\xc3i\x00\x00\x00\x05a,asrye,asa dnlrml a  drsed l  aiprl

Aplicamos la destransformada de Burrows-Wheeler

In [22]:

def insert_byte(headquarter,string):
    string = bytearray(string)
    if len(headquarter) == 0:
        for i in range(len(string)):
            headquarter.append([string[i]])        
    else:
        for i in range(len(string)):
            headquarter[i].insert(0, string[i])
    return headquarter

def sort_headquarters(headquarters):
  heapq.heapify(headquarters)
  return [heapq.heappop(headquarters) for _ in range(len(headquarters))]

def convert_to_bytes(string):
    return bytes(string)

def reverse_bwt(block):
    headquarter = list()
    bloque = block[0]
    for i in range(len(bloque)):
        bloque = insert_byte(headquarter,block[0])
        headquarter=sort_headquarters(headquarter)
    return (headquarter[block[1]])

def save_file(reverse_file, filename):
    with open (filename.replace(".bwt","").replace("/datos","/datos/descompress"),"ab") as file:
        file.write(bytes(reverse_file))
    file.close()
def get_blocks(data,segment_size):
    bloques=int(((len(data)))/(BYTESVENTANA+TAMAÑO_BLOQUE))
    blocks=[]
    for i in range(bloques):
        index=int.from_bytes(data[i*(BYTESVENTANA+TAMAÑO_BLOQUE):i*(BYTESVENTANA+TAMAÑO_BLOQUE)+BYTESVENTANA], "big")
        block=data[i*(BYTESVENTANA+TAMAÑO_BLOQUE)+BYTESVENTANA:i*(BYTESVENTANA+TAMAÑO_BLOQUE)+BYTESVENTANA+TAMAÑO_BLOQUE]
        bloque_tupla = (block,index)
        blocks.append(bloque_tupla)
    return blocks
def revert_bwt(filename):
    with open(filename,"rb") as f:
        data = f.read()
        cut_point=int.from_bytes(data[:BYTESVENTANA], "big")
        blocks= get_blocks(data[BYTESVENTANA:],TAMAÑO_BLOQUE)
        print(blocks)
        for i in range(len(blocks)):
            block = blocks[i]
            clear_output(wait=True)
            print(i,":",len(blocks),"     ",((i+1)*TAMAÑO_BLOQUE/len(data))*100,"%")
            if i < len(blocks):
                reverse=reverse_bwt(block)
            else:
                block=block[:cut_point]
                print("ultimo bloque: ",block)
                reverse=reverse_bwt(block)
            print(bytes(reverse))
            save_file(reverse,filename)
        f.close()   
# Invertimos la cadena de texto.
filename = "../datos/texto1.txt.bwt"
revert_bwt(filename)

213 : 214       92.27452389507725 %
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00m\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xad\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00.'


In [17]:
with open("../datos/descompress/texto1.txt","rb") as f:
    data = f.read()
    print(data)

b'Durante mucho tiempo, 6 clanes alrededor del mundo hab\xc3\xadan librado grandes batallas entre s\xc3\xad, pero un d\xc3\xada todo eso termin\xc3\xb3. Una niebla p\xc3\xbarpura se propag\xc3\xb3 por todos los rincones del planeta corrompiendo el alma de todos aquellos que tocaba, esta peste fue nombrada \xe2\x80\x9cBlinded Shadows\xe2\x80\x9d y en un intento desesperado por sobrevivir los humanos, elfos, cricetas, terralis, demonios y dioses dejaron a un lado sus diferencias para unirse en una sola naci\xc3\xb3n, esta nueva fuerza fue llamada Aurora, por la reina de los dioses, cuyo poder era tan grande que fue capaz de crear un lugar seguro de Blinded Shadows. Debido a sus grandes haza\xc3\xb1as, fue nombrada reina suprema y se catalog\xc3\xb3 su poder como el de un ser supremo, trascendiendo las cualidades de un simple dios. Un ser supremo era alguien omnipresente, omnisciente y omnipotente, pero a pesar de tales poderes, Blinded Shadows hab\xc3\xada sido creada por un poder de la 